In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Instalar as dependencias

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget -q https://downloads.apache.org/spark/spark-2.4.8/spark-2.4.8-bin-hadoop2.7.tgz

In [ ]:
!tar xf spark-2.4.8-bin-hadoop2.7.tgz

In [ ]:
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.8-bin-hadoop2.7"

# Criar uma sessao Spark

In [ ]:
import findspark
# PySpark isn't on sys.path by default
# Findspark adds pyspark to sys.path at runtime in order to locate spark in the system
findspark.init()
from pyspark.sql import SparkSession
# local session
spark = SparkSession.builder.master("local[*]").config('spark.ui.port', '4050').getOrCreate()
spark

# Configuraçao do Spark UI

In [ ]:
# ngrok is a cross-platform application that enables developers to expose a local development server to the Internet with minimal effort
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')

--2021-10-03 14:33:38--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.161.241.46, 18.205.222.128, 54.237.133.81, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.161.241.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  5.49MB/s    in 2.4s    

2021-10-03 14:33:41 (5.49 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [ ]:
!curl -s http://localhost:4040/api/tunnels

{"tunnels":[{"name":"command_line","uri":"/api/tunnels/command_line","public_url":"https://3f96-130-211-226-165.ngrok.io","proto":"https","config":{"addr":"http://localhost:4050","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}},{"name":"command_line (http)","uri":"/api/tunnels/command_line%20%28http%29","public_url":"http://3f96-130-211-226-165.ngrok.io","proto":"http","config":{"addr":"http://localhost:4050","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}}],"uri":"/api/tunnels"}


Agora você deve ser capaz de visualizar os trabalhos e suas etapas no link criado "public_url".

# Ler um dataset

## Exemplo simples

In [ ]:
l = [('Alice', 1), ('Pedro', 30)]

In [ ]:
df = spark.createDataFrame(l, ['name', 'age'])

In [ ]:
df.show()

+-----+---+
| name|age|
+-----+---+
|Alice|  1|
|Pedro| 30|
+-----+---+



In [ ]:
df.select("age").where("age < 20").show()

+---+
|age|
+---+
|  1|
+---+



In [ ]:
df.registerTempTable("pessoa")

In [ ]:
df2 = spark.sql("select age from pessoa where age < 20")

In [ ]:
df2.show()

+---+
|age|
+---+
|  1|
+---+



## Outro Exemplo

**crim**
per capita crime rate by town.

**zn**
proportion of residential land zoned for lots over 25,000 sq.ft.

**indus**
proportion of non-retail business acres per town.

**chas**
Charles River dummy variable (= 1 if tract bounds river; 0 otherwise).

**nox**
nitrogen oxides concentration (parts per 10 million).

**rm**
average number of rooms per dwelling.

**age**
proportion of owner-occupied units built prior to 1940.

**dis**
weighted mean of distances to five Boston employment centres.

**rad**
index of accessibility to radial highways.

**tax**
full-value property-tax rate per \$10,000.

**ptratio**
pupil-teacher ratio by town.

**black**
1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town.

**lstat**
lower status of the population (percent).

**medv**
median value of owner-occupied homes in \$1000s.


In [ ]:
from google.colab import files
files.upload()
!ls

Saving BostonHousing.csv to BostonHousing.csv
BostonHousing.csv	      sample_data		     spark-warehouse
ngrok			      spark-2.4.8-bin-hadoop2.7
ngrok-stable-linux-amd64.zip  spark-2.4.8-bin-hadoop2.7.tgz


#### Exemplo simples

In [ ]:
df = spark.read.csv("/content/BostonHousing.csv", header=True, inferSchema=True)
df.show()

+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+
|   crim|  zn|indus|chas|  nox|   rm|  age|   dis|rad|tax|ptratio|     b|lstat|medv|
+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+
|0.00632|18.0| 2.31|   0|0.538|6.575| 65.2|  4.09|  1|296|   15.3| 396.9| 4.98|24.0|
|0.02731| 0.0| 7.07|   0|0.469|6.421| 78.9|4.9671|  2|242|   17.8| 396.9| 9.14|21.6|
|0.02729| 0.0| 7.07|   0|0.469|7.185| 61.1|4.9671|  2|242|   17.8|392.83| 4.03|34.7|
|0.03237| 0.0| 2.18|   0|0.458|6.998| 45.8|6.0622|  3|222|   18.7|394.63| 2.94|33.4|
|0.06905| 0.0| 2.18|   0|0.458|7.147| 54.2|6.0622|  3|222|   18.7| 396.9| 5.33|36.2|
|0.02985| 0.0| 2.18|   0|0.458| 6.43| 58.7|6.0622|  3|222|   18.7|394.12| 5.21|28.7|
|0.08829|12.5| 7.87|   0|0.524|6.012| 66.6|5.5605|  5|311|   15.2| 395.6|12.43|22.9|
|0.14455|12.5| 7.87|   0|0.524|6.172| 96.1|5.9505|  5|311|   15.2| 396.9|19.15|27.1|
|0.21124|12.5| 7.87|   0|0.524|5.631|100.0|6.0821|  5|311|   15.2

In [ ]:
df.dtypes

[('crim', 'double'),
 ('zn', 'double'),
 ('indus', 'double'),
 ('chas', 'int'),
 ('nox', 'double'),
 ('rm', 'double'),
 ('age', 'double'),
 ('dis', 'double'),
 ('rad', 'int'),
 ('tax', 'int'),
 ('ptratio', 'double'),
 ('b', 'double'),
 ('lstat', 'double'),
 ('medv', 'double')]

In [ ]:
df.groupby("age").avg("crim").show()

+----+-----------------+
| age|        avg(crim)|
+----+-----------------+
|74.5|          0.10153|
|56.8|          0.07151|
|98.3|          3.85469|
|78.9|          0.02731|
|15.7|          0.02177|
|67.0|          0.57834|
|96.8|          5.66998|
|56.5|          0.62739|
|32.3|          0.14052|
|49.9|          0.35114|
|54.2|          0.06905|
|98.1|         9.031795|
|86.9|          0.61154|
|83.5|          0.11132|
|94.9|          2.73397|
|90.3|          0.67191|
|97.1|          0.12802|
|65.4|          7.83932|
|59.5|          0.01951|
|76.5|4.179653333333333|
+----+-----------------+
only showing top 20 rows



In [ ]:
df.registerTempTable("pessoa")

In [ ]:
spark.sql("select age, avg(crim) from pessoa group by age").show()

+----+-----------------+
| age|        avg(crim)|
+----+-----------------+
|74.5|          0.10153|
|56.8|          0.07151|
|98.3|          3.85469|
|78.9|          0.02731|
|15.7|          0.02177|
|67.0|          0.57834|
|96.8|          5.66998|
|56.5|          0.62739|
|32.3|          0.14052|
|49.9|          0.35114|
|54.2|          0.06905|
|98.1|         9.031795|
|86.9|          0.61154|
|83.5|          0.11132|
|94.9|          2.73397|
|90.3|          0.67191|
|97.1|          0.12802|
|65.4|          7.83932|
|59.5|          0.01951|
|76.5|4.179653333333333|
+----+-----------------+
only showing top 20 rows



#### Usando a sintaxe do SQL

In [ ]:
df.registerTempTable("boston")

In [ ]:
df.summary().show()

+-------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+
|summary|              crim|                zn|             indus|              chas|                nox|                rm|               age|              dis|              rad|               tax|           ptratio|                 b|             lstat|              medv|
+-------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+
|  count|               506|               506|               506|               506|                506|               506|               506|              506|              

In [ ]:
df2 = df.where("age < 20").groupBy("age").avg("crim").orderBy(df.age.asc())
df2.show()

+----+---------+
| age|avg(crim)|
+----+---------+
| 2.9|  0.12744|
| 6.0|  0.07896|
| 6.2|  0.19539|
| 6.5|  0.15936|
| 6.6|  0.11488|
| 6.8|  0.08221|
| 7.8| 0.109605|
| 8.4|  0.36894|
| 8.9|  0.21409|
| 9.8|  0.28955|
| 9.9|  0.02187|
|10.0|  0.05561|
|13.0|   0.1403|
|13.9|  0.01778|
|14.7|  0.19186|
|15.3|   0.0315|
|15.7|  0.02177|
|15.8|  0.03359|
|16.3|  0.16211|
|17.0|  0.29819|
+----+---------+
only showing top 20 rows



In [ ]:
df2 = spark.sql("select age, avg(crim) from boston where age < 20 group by age order by age asc")
df2.show()

+----+---------+
| age|avg(crim)|
+----+---------+
| 2.9|  0.12744|
| 6.0|  0.07896|
| 6.2|  0.19539|
| 6.5|  0.15936|
| 6.6|  0.11488|
| 6.8|  0.08221|
| 7.8| 0.109605|
| 8.4|  0.36894|
| 8.9|  0.21409|
| 9.8|  0.28955|
| 9.9|  0.02187|
|10.0|  0.05561|
|13.0|   0.1403|
|13.9|  0.01778|
|14.7|  0.19186|
|15.3|   0.0315|
|15.7|  0.02177|
|15.8|  0.03359|
|16.3|  0.16211|
|17.0|  0.29819|
+----+---------+
only showing top 20 rows



In [ ]:
df2.explain()

== Physical Plan ==
*(3) Sort [age#38 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(age#38 ASC NULLS FIRST, 200)
   +- *(2) HashAggregate(keys=[age#38], functions=[avg(crim#32)])
      +- Exchange hashpartitioning(age#38, 200)
         +- *(1) HashAggregate(keys=[age#38], functions=[partial_avg(crim#32)])
            +- *(1) Project [crim#32, age#38]
               +- *(1) Filter (isnotnull(age#38) && (age#38 < 20.0))
                  +- *(1) FileScan csv [crim#32,age#38] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/content/BostonHousing.csv], PartitionFilters: [], PushedFilters: [IsNotNull(age), LessThan(age,20.0)], ReadSchema: struct<crim:double,age:double>


# Exemplo Pratico - Como Funciona?

**parallelize**: Distribute a local Python collection to form an RDD. Using xrange is recommended if the input represents a range for performance

In [ ]:
spark

In [ ]:
sc = spark.sparkContext

### Word count

In [ ]:
from google.colab import files
files.upload()
!ls

Saving word_count_ex.txt to word_count_ex.txt
BostonHousing.csv	      sample_data		     spark-warehouse
ngrok			      spark-2.4.8-bin-hadoop2.7      word_count_ex.txt
ngrok-stable-linux-amd64.zip  spark-2.4.8-bin-hadoop2.7.tgz


In [ ]:
text_file = sc.textFile("word_count_ex.txt")

In [ ]:
text_file

word_count_ex.txt MapPartitionsRDD[102] at textFile at NativeMethodAccessorImpl.java:0

In [ ]:
counts = text_file.flatMap(lambda line: line.split(" ")) \
             .map(lambda word: (word, 1)) \
             .reduceByKey(lambda a, b: a + b)

In [ ]:
counts

PythonRDD[107] at RDD at PythonRDD.scala:53

In [ ]:
counts.collect()

[('Aine', 3), ('Amy', 1), ('Iwona', 4), ('Canan', 2)]

In [ ]:
counts.saveAsTextFile("res_word_count_ex.txt")

In [ ]:
# print(counts.toDebugString().decode())

### Transformaçoes a Açoes

In [ ]:
rdd.map(lambda x: (x, 1))

PythonRDD[113] at RDD at PythonRDD.scala:53

In [ ]:
rdd.map(lambda x: (x, 1)).collect()

[('b', 1), ('a', 1), ('c', 1)]

In [ ]:
# map
rdd = sc.parallelize(["b", "a", "c"])
sorted(rdd.map(lambda x: (x, 1)).collect())

[('a', 1), ('b', 1), ('c', 1)]

In [ ]:
# filter
rdd = sc.parallelize([1, 2, 3, 4, 5])
rdd.filter(lambda x: x % 2 == 0).collect()

sc.parallelize(range(100), 100).filter(lambda x: x > 90).take(3)

In [ ]:
# Açoes
# reduce
from operator import add
sc.parallelize([1, 2, 3, 4, 5]).reduce(add)

15

In [ ]:
# reduce
sc.parallelize((2 for _ in range(10))).map(lambda x: 1).cache().reduce(add)

# count
sc.parallelize([2, 3, 4]).count()

# collect
sc.parallelize([1, 2, 3, 4, 5], 3).glom().collect()

In [ ]:
# TODO
# https://spark.apache.org/docs/2.4.0/api/python/pyspark.html